In [4]:
import pickle
from pathlib import Path

import pandas as pd

# ------------------------------------------------------------------
# 1. chemin du pickle (adapter si besoin)
PKL = Path("C:/Users/Dell/projects/regional_flex/results/full_year.pkl")

# 2. chargement
with PKL.open("rb") as f:
    res = pickle.load(f)

# ------------------------------------------------------------------
# 3. reconstruction d’un DataFrame « long » ⇢ « large »
#    (index = pas de temps, colonnes = variables)
first_var = next(iter(res["variables"].values()))
idx = pd.date_range("2022-01-01", periods=len(first_var), freq="30min")

table = {}
for var_name, series_dict in res["variables"].items():
    # Series_dict : {t -> valeur}
    # on le remet au même index complet, les manquants ⇢ 0
    s = pd.Series(series_dict).reindex(range(len(idx)), fill_value=0.0)
    table[var_name] = s.values

df = pd.DataFrame(table, index=idx)

# ------------------------------------------------------------------
# 4. aperçu
print(df.shape)
df.head()

df.to_csv("C:/Users/Dell/projects/regional_flex/full_year_results.csv")


(17473, 76)


In [5]:
df.head()

,dispatch_hydro_Auvergne_Rhone_Alpes,dispatch_nuclear_Auvergne_Rhone_Alpes,dispatch_thermal_gas_Auvergne_Rhone_Alpes,dispatch_thermal_coal_Auvergne_Rhone_Alpes,dispatch_biofuel_Auvergne_Rhone_Alpes,storage_soc_STEP_Auvergne_Rhone_Alpes,storage_charge_STEP_Auvergne_Rhone_Alpes,storage_discharge_STEP_Auvergne_Rhone_Alpes,storage_soc_batteries_Auvergne_Rhone_Alpes,storage_charge_batteries_Auvergne_Rhone_Alpes,...,flow_out_Auvergne_Rhone_Alpes_Provence_Alpes_Cote_dAzur,flow_out_Nouvelle_Aquitaine_Auvergne_Rhone_Alpes,flow_out_Nouvelle_Aquitaine_Occitanie,flow_out_Nouvelle_Aquitaine_Provence_Alpes_Cote_dAzur,flow_out_Occitanie_Auvergne_Rhone_Alpes,flow_out_Occitanie_Nouvelle_Aquitaine,flow_out_Occitanie_Provence_Alpes_Cote_dAzur,flow_out_Provence_Alpes_Cote_dAzur_Auvergne_Rhone_Alpes,flow_out_Provence_Alpes_Cote_dAzur_Nouvelle_Aquitaine,flow_out_Provence_Alpes_Cote_dAzur_Occitanie
2022-01-01 00:00:00,6260.024,2112.650,0.0,0.0,0.0,30550.0,0.0,0.0000,100.0,0.0,...,704.6740,0.0,0.0,1236.29780,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 00:30:00,6157.397,2105.835,0.0,0.0,0.0,30550.0,0.0,0.0000,100.0,0.0,...,653.2320,0.0,0.0,873.16871,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 01:00:00,6054.770,2099.020,0.0,0.0,0.0,30550.0,0.0,0.0000,0.0,0.0,...,653.7900,0.0,0.0,990.84371,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 01:30:00,5952.143,2092.205,0.0,0.0,0.0,30550.0,0.0,0.0000,0.0,0.0,...,533.3480,0.0,0.0,1122.46460,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 02:00:00,5849.516,2085.390,0.0,0.0,0.0,30550.0,0.0,59.3161,0.0,0.0,...,483.2221,0.0,0.0,939.69371,0.0,0.0,0.0,0.0,0.0,0.0
